not simple linear regression
    2 or more models
    

In [1]:
import pandas as pd
import statsmodels.formula.api as stats
import statsmodels.api as sm
import numpy as np
import math
import sklearn.metrics as sk
from bs4 import BeautifulSoup
import requests
import json
from os import path

In [2]:
not path.exists('FullStats\FullSeasonStats_2010.csv')

False

In [98]:
for key in bigStats:
    print(key+str(len(bigStats[key])))

Name_HigherRank5386
GamesPlayed_HigherRank5386
ConfWins_HigherRank5386
ConfLosses_HigherRank5386
TotalWins_HigherRank5386
TotalLosses_HigherRank5386
OffEff_HigherRank5386
DefEff_HigherRank5386
Turnovers_HigherRank5386
ForcedTurnovers_HigherRank5386
2PFGOff_HigherRank5386
2PFGDef_HigherRank5386
3PFGOff_HigherRank5386
3PFGDef_HigherRank5386
FTOff_HigherRank5386
FTDef_HigherRank5386
Rank_HigherRank26930
Name_LowerRank5386
GamesPlayed_LowerRank5386
OffEff_LowerRank5386
DefEff_LowerRank5386
Turnovers_LowerRank5386
ForcedTurnovers_LowerRank5386
2PFGOff_LowerRank5386
2PFGDef_LowerRank5386
3PFGOff_LowerRank5386
3PFGDef_LowerRank5386
FTOff_LowerRank5386
FTDef_LowerRank5386
Rank_LowerRank5386
Results_HigherRank5386
Differntial5386
Score5386
PlusMinus_HigherRank5386


In [ ]:
years = list(range(2010,2022))
for year in years:
    GamesPlayed=[]
    bigStats = {}
    filename = 'FullStats\FullSeasonStats_'+str(year)+'.csv'
    with open(filename,'r') as f:
        df = pd.read_csv(f)
    for team in df.Name:
        filename2 = 'Games\\'+str(year)+'\\'+team+'_'+str(year)+'_Opponent.csv'
        with open(filename2,'r') as f:
            df2 = pd.read_csv(f)
            num=0
        for opponent in df2.Opponent:
            if opponent not in GamesPlayed:
                HigherRank = GetStats(team,df)
                LowerRank = GetStats(opponent,df)
                if not LowerRank.empty:
                    AppendNewStats(HigherRank,LowerRank,bigStats)
                    AppendGameResults(df2,num,bigStats)
            num+=1
        GamesPlayed.append(team)
    bigStats1 = pd.DataFrame(data = bigStats)
    bigStats1.to_csv('Matchup\\MatchupStats_'+str(year)+'.csv', index = False)
    print('Finished Processing '+str(year))


Finished Processing 2010
Finished Processing 2011
Finished Processing 2012
Finished Processing 2013


In [103]:
def AppendNewStats(HigherRank,LowerRank,bigStats):
    record = ['ConfRecord','TotalRecord']
    for item in HigherRank:
        keyval1 = item+'_HigherRank'
        if item in record:
            keyvalWin = keyval1.replace('Record','Wins')
            keyvalLoss = keyval1.replace('Record','Losses')
            ratio = HigherRank[item].iloc[0].split('_')
            append_value(bigStats, keyvalWin, ratio[0])
            append_value(bigStats, keyvalLoss, ratio[1])
        else:
            append_value(bigStats, keyval1, HigherRank[item].iloc[0])
    for item in LowerRank:
        keyval2 = item+'_LowerRank'
        if item in record:
            keyvalWin = keyval2.replace('Record','Wins')
            keyvalLoss = keyval2.replace('Record','Losses')
            ratio = LowerRank[item].iloc[0].split('_')
            append_value(bigStats, keyvalWin, ratio[0])
            append_value(bigStats, keyvalLoss, ratio[1])
        else:
            append_value(bigStats, keyval2, LowerRank[item].iloc[0])
    return(0)

def GetStats(team,fullstats):
    return(fullstats[fullstats['Name'] == team])
    
def AppendGameResults(df2,num,bigStats):
    for item in df2:
        if item != 'Opponent':
            if item == 'Score':
                scores = df2[item].iloc[num].split('_')
                for i in range(0,2):
                    scores[i] = (scores[i].split())[0]
                differential = int(scores[1])-int(scores[0])
                if df2['Results'].iloc[num] == 'W':
                    differential = abs(differential)
                append_value(bigStats, 'Differntial', differential)
            if item == 'Results' or item == 'PlusMinus':
                keyval = item+'_HigherRank'
            else:
                keyval = item
            append_value(bigStats, keyval, df2[item].iloc[num])
    return(0)
    
    
stats = {}
filename = 'FullStats\FullSeasonStats_'+str(year)+'.csv'
with open(filename,'r') as f:
    df = pd.read_csv(f)
played = ['Villanova','Gonzaga']
for name in df.Name:
    if name in played:
        df_new = df[df['Name'] == name]
        for item in df_new:
            append_value(stats, item, df_new[item].iloc[0])
            

In [87]:
def append_value(dict_obj, key, value):
    # Check if key exist in dict or not
    if key in dict_obj:
        # Key exist in dict.
        # Check if type of value of key is list or not
        if not isinstance(dict_obj[key], list):
            # If type is not list then make it list
            dict_obj[key] = [dict_obj[key]]
        # Append the value in list
        dict_obj[key].append(value)
    else:
        # As key is not in dict,
        # so, add key-value pair
        dict_obj[key] = value

In [95]:
#First Regression Attempt

filename = 'Matchup\\MatchupStats_2010.csv'
with open(filename,'r') as f:
    df = pd.read_csv(f)
print(df.head())
#lm_fit = stats.ols(formula='X2~X1+X3+X4+X5+X6',data=df).fit()
#lm_fit.summary()


  Name_HigherRank  GamesPlayed_HigherRank ConfRecord_HigherRank  \
0            Duke                      40                  13_3   
1            Duke                      40                  13_3   
2            Duke                      40                  13_3   
3            Duke                      40                  13_3   
4            Duke                      40                  13_3   

  TotalRecord_HigherRank  OffEff_HigherRank  DefEff_HigherRank  \
0                   35_5              120.8               87.8   
1                   35_5              120.8               87.8   
2                   35_5              120.8               87.8   
3                   35_5              120.8               87.8   
4                   35_5              120.8               87.8   

   Turnovers_HigherRank  ForcedTurnovers_HigherRank  2PFGOff_HigherRank  \
0                  16.5                        21.3                46.9   
1                  16.5                        21.